tgb - 11/13/2019 - Continuity of 028 but for simultaneous training while files are pre-processing

# 0) Imports

In [1]:
from cbrain.imports import *
from cbrain.data_generator import *
from cbrain.cam_constants import *
from cbrain.losses import *
from cbrain.utils import limit_mem
from cbrain.layers import *
from cbrain.data_generator import DataGenerator
import tensorflow as tf
import tensorflow.math as tfm
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
import xarray as xr
import numpy as np
from cbrain.model_diagnostics import ModelDiagnostics
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.image as imag
import scipy.integrate as sin

TRAINDIR = '/local/Tom.Beucler/SPCAM_PHYS/'
DATADIR = '/project/meteo/w2w/A6/S.Rasp/SP-CAM/fluxbypass_aqua/'
PREFIX = '8col009_01_'
%cd /filer/z-sv-pool12c/t/Tom.Beucler/SPCAM/CBRAIN-CAM

# Otherwise tensorflow will use ALL your GPU RAM for no reason
limit_mem()

/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages

/home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/notebooks/tbeucler_devlog
/filer/z-sv-pool12c/t/Tom.Beucler/SPCAM/CBRAIN-CAM


# 1) NN with only q and T as inputs

## 1.1) Rescaling

In [2]:
scale_dict = {
    'PHQ': L_V/G,  
    'TPHYSTND': C_P/G, 
    'FSNT': 1, 
    'FSNS': 1, 
    'FLNT': 1, 
    'FLNS': 1, 
}

In [3]:
# Takes representative value for PS since purpose is normalization
PS = 1e5; P0 = 1e5;
P = P0*hyai+PS*hybi; # Total pressure [Pa]
dP = P[1:]-P[:-1]; # Differential pressure [Pa]

In [4]:
for v in ['PHQ','TPHYSTND']:
    scale_dict[v] *= dP

In [6]:
save_pickle('./nn_config/scale_dicts/100_POG_scaling.pkl', scale_dict)

In [20]:
in_vars = ['QBP','TBP','PS', 'SOLIN', 'SHFLX', 'LHFLX']
out_vars = ['PHQ','TPHYSTND','FSNT','FSNS','FLNT','FLNS']
train_gen = DataGenerator(
    data_fn = '/local/Tom.Beucler/SPCAM_PHYS/101_train_shuffle.nc',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = '/local/Tom.Beucler/SPCAM_PHYS/100_norm.nc',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True
)

OSError: [Errno -101] NetCDF: HDF error: b'/local/Tom.Beucler/SPCAM_PHYS/101_train_shuffle.nc'

In [22]:
valid_gen = DataGenerator(
    data_fn = '/local/Tom.Beucler/SPCAM_PHYS/101_valid.nc',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = '/local/Tom.Beucler/SPCAM_PHYS/100_norm.nc',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True
)

In [24]:
X, Y = valid_gen[0]; X.shape, Y.shape

((1024, 64), (1024, 64))

## 1.2) Model and training

In [14]:
inp = Input(shape=(64,))
densout = Dense(128, activation='linear')(inp)
densout = LeakyReLU(alpha=0.3)(densout)
for i in range (5):
    densout = Dense(128, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
densout = Dense(64, activation='linear')(densout)
out = LeakyReLU(alpha=0.3)(densout)
NNmodel = tf.keras.models.Model(inp,out)

In [15]:
NNmodel.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense_10 (Dense)             (None, 128)               8320      
_________________________________________________________________
leaky_re_lu_10 (LeakyReLU)   (None, 128)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 128)               16512     
_________________________________________________________________
leaky_re_lu_11 (LeakyReLU)   (None, 128)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 128)               16512     
_________________________________________________________________
leaky_re_lu_12 (LeakyReLU)   (None, 128)               0   

In [27]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint('/local/Tom.Beucler/SPCAM_PHYS/HDF5_DATA/POG101.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [28]:
NNmodel.compile(tf.keras.optimizers.Adam(),loss=mse)

Trained for 15 epochs in total

In [30]:
Nep = 5
NNmodel.fit_generator(train_gen, epochs=Nep, validation_data=valid_gen,callbacks=[earlyStopping,mcp_save])

Epoch 1/5
41376/41376 [==============================] - 574s 14ms/step - loss: 498.8082 - val_loss: 503.0716
Epoch 2/5
41376/41376 [==============================] - 497s 12ms/step - loss: 497.8446 - val_loss: 505.1086
Epoch 3/5
41376/41376 [==============================] - 548s 13ms/step - loss: 496.8365 - val_loss: 507.1944
Epoch 4/5
41376/41376 [==============================] - 522s 13ms/step - loss: 496.1410 - val_loss: 502.1063
Epoch 5/5
41376/41376 [==============================] - 446s 11ms/step - loss: 495.3583 - val_loss: 507.9743


# 2) NN with only RH and T as inputs

## 2.1) Rescaling

In [31]:
in_vars = ['RH','TBP','PS', 'SOLIN', 'SHFLX', 'LHFLX']
out_vars = ['PHQ','TPHYSTND','FSNT','FSNS','FLNT','FLNS']
train_gen = DataGenerator(
    data_fn = '/local/Tom.Beucler/SPCAM_PHYS/104_train_shuffle.nc',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = '/local/Tom.Beucler/SPCAM_PHYS/103_norm.nc',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True
)

In [32]:
valid_gen = DataGenerator(
    data_fn = '/local/Tom.Beucler/SPCAM_PHYS/104_valid.nc',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = '/local/Tom.Beucler/SPCAM_PHYS/103_norm.nc',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True
)

In [33]:
X, Y = valid_gen[0]; X.shape, Y.shape

((1024, 64), (1024, 64))

## 2.2) Model and training

In [34]:
inp2 = Input(shape=(64,))
densout = Dense(128, activation='linear')(inp2)
densout = LeakyReLU(alpha=0.3)(densout)
for i in range (5):
    densout = Dense(128, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
densout = Dense(64, activation='linear')(densout)
out2 = LeakyReLU(alpha=0.3)(densout)
NNmodel2 = tf.keras.models.Model(inp2,out2)

In [35]:
NNmodel2.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense_17 (Dense)             (None, 128)               8320      
_________________________________________________________________
leaky_re_lu_17 (LeakyReLU)   (None, 128)               0         
_________________________________________________________________
dense_18 (Dense)             (None, 128)               16512     
_________________________________________________________________
leaky_re_lu_18 (LeakyReLU)   (None, 128)               0         
_________________________________________________________________
dense_19 (Dense)             (None, 128)               16512     
_________________________________________________________________
leaky_re_lu_19 (LeakyReLU)   (None, 128)               0   

In [36]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint('/local/Tom.Beucler/SPCAM_PHYS/HDF5_DATA/POG104.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [37]:
NNmodel2.compile(tf.keras.optimizers.Adam(),loss=mse)

In [38]:
Nep = 15
NNmodel2.fit_generator(train_gen, epochs=Nep, validation_data=valid_gen,callbacks=[earlyStopping,mcp_save])

Epoch 1/15
41376/41376 [==============================] - 300s 7ms/step - loss: 587.5173 - val_loss: 553.5788
Epoch 2/15
41376/41376 [==============================] - 345s 8ms/step - loss: 529.7140 - val_loss: 532.6032
Epoch 3/15
41376/41376 [==============================] - 509s 12ms/step - loss: 516.5800 - val_loss: 520.6585
Epoch 4/15
41376/41376 [==============================] - 535s 13ms/step - loss: 509.5971 - val_loss: 519.6243
Epoch 5/15
41376/41376 [==============================] - 612s 15ms/step - loss: 505.0548 - val_loss: 513.3901
Epoch 6/15
41376/41376 [==============================] - 627s 15ms/step - loss: 501.8029 - val_loss: 508.0100
Epoch 7/15
41376/41376 [==============================] - 568s 14ms/step - loss: 499.4309 - val_loss: 505.2136
Epoch 8/15
41376/41376 [==============================] - 590s 14ms/step - loss: 497.4045 - val_loss: 508.2901
Epoch 9/15
41376/41376 [==============================] - 625s 15ms/step - loss: 495.8311 - val_loss: 502.9355
Epo

KeyboardInterrupt: 

# 3) NN with only QBP and TfromMA as inputs

## 3.1) Rescaling

In [11]:
import time
time.sleep(1800)

In [10]:
in_vars = ['QBP','TfromMA','PS', 'SOLIN', 'SHFLX', 'LHFLX']
out_vars = ['PHQ','TPHYSTND','FSNT','FSNS','FLNT','FLNS']
train_gen = DataGenerator(
    data_fn = '/local/Tom.Beucler/SPCAM_PHYS/107_train_shuffle.nc',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = '/local/Tom.Beucler/SPCAM_PHYS/106_norm.nc',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True
)

In [13]:
valid_gen = DataGenerator(
    data_fn = '/local/Tom.Beucler/SPCAM_PHYS/107_valid.nc',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = '/local/Tom.Beucler/SPCAM_PHYS/106_norm.nc',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True
)

In [14]:
X, Y = train_gen[0]; X.shape, Y.shape

((1024, 64), (1024, 64))

## 3.2) Model and training

In [15]:
inp3 = Input(shape=(64,))
densout = Dense(128, activation='linear')(inp3)
densout = LeakyReLU(alpha=0.3)(densout)
for i in range (5):
    densout = Dense(128, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
densout = Dense(64, activation='linear')(densout)
out3 = LeakyReLU(alpha=0.3)(densout)
NNmodel3 = tf.keras.models.Model(inp3,out3)

W1117 03:29:17.228912 22808143263552 deprecation.py:506] From /home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [20]:
NNmodel3.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense (Dense)                (None, 128)               8320      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 128)               0     

In [17]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint('/local/Tom.Beucler/SPCAM_PHYS/HDF5_DATA/POG107.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [18]:
NNmodel3.compile(tf.keras.optimizers.Adam(),loss=mse)

In [19]:
Nep = 15
NNmodel3.fit_generator(train_gen, epochs=Nep, validation_data=valid_gen,callbacks=[earlyStopping,mcp_save])

Epoch 1/15
41376/41376 [==============================] - 324s 8ms/step - loss: 624.8763 - val_loss: 581.0365
Epoch 2/15
41376/41376 [==============================] - 324s 8ms/step - loss: 554.6878 - val_loss: 558.9215
Epoch 3/15
41376/41376 [==============================] - 327s 8ms/step - loss: 537.8437 - val_loss: 549.2132
Epoch 4/15
41376/41376 [==============================] - 322s 8ms/step - loss: 528.8062 - val_loss: 534.4663
Epoch 5/15
41376/41376 [==============================] - 324s 8ms/step - loss: 523.5654 - val_loss: 531.3699
Epoch 6/15
41376/41376 [==============================] - 324s 8ms/step - loss: 519.5813 - val_loss: 527.4305
Epoch 7/15
41376/41376 [==============================] - 336s 8ms/step - loss: 516.6053 - val_loss: 521.9055
Epoch 8/15
41376/41376 [==============================] - 320s 8ms/step - loss: 514.3161 - val_loss: 520.4829
Epoch 9/15
41376/41376 [==============================] - 318s 8ms/step - loss: 512.2378 - val_loss: 522.5594
Epoch 10/1

# 4) NN using only RH and TfromMA as inputs

## 4.1) Preprocessing

In [24]:
!python preprocessing-11132019.py -c /home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/pp_config/109_POG_RHandTfromMAinput_test.yml

/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages

In [25]:
!python preprocessing-11132019.py -c /home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/pp_config/110_POG_84G_RHandTfromMAinput.yml

/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages

## 4.2) Rescaling

In [26]:
in_vars = ['RH','TfromMA','PS', 'SOLIN', 'SHFLX', 'LHFLX']
out_vars = ['PHQ','TPHYSTND','FSNT','FSNS','FLNT','FLNS']
train_gen = DataGenerator(
    data_fn = '/local/Tom.Beucler/SPCAM_PHYS/110_train_shuffle.nc',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = '/local/Tom.Beucler/SPCAM_PHYS/109_norm.nc',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True
)

In [27]:
valid_gen = DataGenerator(
    data_fn = '/local/Tom.Beucler/SPCAM_PHYS/110_valid.nc',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = '/local/Tom.Beucler/SPCAM_PHYS/109_norm.nc',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True
)

In [28]:
X, Y = train_gen[0]; X.shape, Y.shape

((1024, 64), (1024, 64))

## 4.3) Model and training

In [29]:
inp4 = Input(shape=(64,))
densout = Dense(128, activation='linear')(inp4)
densout = LeakyReLU(alpha=0.3)(densout)
for i in range (5):
    densout = Dense(128, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
densout = Dense(64, activation='linear')(densout)
out4 = LeakyReLU(alpha=0.3)(densout)
NNmodel4 = tf.keras.models.Model(inp4,out4)

In [30]:
NNmodel4.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense_14 (Dense)             (None, 128)               8320      
_________________________________________________________________
leaky_re_lu_14 (LeakyReLU)   (None, 128)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 128)               16512     
_________________________________________________________________
leaky_re_lu_15 (LeakyReLU)   (None, 128)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 128)               16512     
_________________________________________________________________
leaky_re_lu_16 (LeakyReLU)   (None, 128)               0   

In [31]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint('/local/Tom.Beucler/SPCAM_PHYS/HDF5_DATA/POG110.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [32]:
NNmodel4.compile(tf.keras.optimizers.Adam(),loss=mse)

In [33]:
Nep = 15
NNmodel4.fit_generator(train_gen, epochs=Nep, validation_data=valid_gen,callbacks=[earlyStopping,mcp_save])

Epoch 1/15
41376/41376 [==============================] - 295s 7ms/step - loss: 581.0607 - val_loss: 545.7116
Epoch 2/15
41376/41376 [==============================] - 290s 7ms/step - loss: 523.3140 - val_loss: 527.0278
Epoch 3/15
41376/41376 [==============================] - 293s 7ms/step - loss: 510.8817 - val_loss: 523.4392
Epoch 4/15
41376/41376 [==============================] - 292s 7ms/step - loss: 504.2300 - val_loss: 521.1177
Epoch 5/15
41376/41376 [==============================] - 293s 7ms/step - loss: 500.0675 - val_loss: 506.5438
Epoch 6/15
41376/41376 [==============================] - 295s 7ms/step - loss: 497.0642 - val_loss: 500.9538
Epoch 7/15
41376/41376 [==============================] - 294s 7ms/step - loss: 494.7105 - val_loss: 504.4769
Epoch 8/15
41376/41376 [==============================] - 296s 7ms/step - loss: 492.8043 - val_loss: 499.8519
Epoch 9/15
41376/41376 [==============================] - 294s 7ms/step - loss: 491.3040 - val_loss: 500.1378
Epoch 10/1

## 4.4) Preprocess +4K

In [5]:
!python preprocessing-11132019.py -c /home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/pp_config/111_POG_84G_p4K_RHandTfromMAinput.yml

/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages

# 5) NN using QBP and Carnotmax as inputs

## 5.1) Rescaling

In [ ]:
in_vars = ['QBP','Carnotmax','PS', 'SOLIN', 'SHFLX', 'LHFLX']
out_vars = ['PHQ','TPHYSTND','FSNT','FSNS','FLNT','FLNS']
train_gen = DataGenerator(
    data_fn = '/local/Tom.Beucler/SPCAM_PHYS/113_train_shuffle.nc',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = '/local/Tom.Beucler/SPCAM_PHYS/112_norm.nc',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True
)

valid_gen = DataGenerator(
    data_fn = '/local/Tom.Beucler/SPCAM_PHYS/113_valid.nc',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = '/local/Tom.Beucler/SPCAM_PHYS/112_norm.nc',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True
)

X, Y = train_gen[12]; X.shape, Y.shape

## 5.2) Model and training

In [15]:
inp5 = Input(shape=(64,))
densout = Dense(128, activation='linear')(inp5)
densout = LeakyReLU(alpha=0.3)(densout)
for i in range (5):
    densout = Dense(128, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
densout = Dense(64, activation='linear')(densout)
out5 = LeakyReLU(alpha=0.3)(densout)
NNmodel5 = tf.keras.models.Model(inp5,out5)

W1115 20:40:38.060026 22499940611904 deprecation.py:506] From /home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [16]:
NNmodel5.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense (Dense)                (None, 128)               8320      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 128)               0     

In [17]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint('/local/Tom.Beucler/SPCAM_PHYS/HDF5_DATA/POG107.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [18]:
NNmodel5.compile(tf.keras.optimizers.Adam(),loss=mse)

In [19]:
Nep = 15
NNmodel5.fit_generator(train_gen, epochs=Nep, validation_data=valid_gen,callbacks=[earlyStopping,mcp_save])

Epoch 1/15
41376/41376 [==============================] - 232s 6ms/step - loss: 631.4662 - val_loss: 586.0113
Epoch 2/15
41376/41376 [==============================] - 245s 6ms/step - loss: 564.7312 - val_loss: 576.2434
Epoch 3/15
41376/41376 [==============================] - 318s 8ms/step - loss: 547.9262 - val_loss: 550.3398
Epoch 4/15
41376/41376 [==============================] - 368s 9ms/step - loss: 539.2624 - val_loss: 548.4072
Epoch 5/15
41376/41376 [==============================] - 337s 8ms/step - loss: 534.1456 - val_loss: 547.2794
Epoch 6/15
41376/41376 [==============================] - 324s 8ms/step - loss: 530.6359 - val_loss: 540.1844
Epoch 7/15
41376/41376 [==============================] - 315s 8ms/step - loss: 527.8472 - val_loss: 537.5466
Epoch 8/15
41376/41376 [==============================] - 234s 6ms/step - loss: 525.5756 - val_loss: 542.6954
Epoch 9/15
41376/41376 [==============================] - 232s 6ms/step - loss: 523.5346 - val_loss: 535.8333
Epoch 10/1

# 6) NN using only Q and (T-TS) as inputs

# 6.1) Rescaling

In [7]:
import time
time.sleep(1800)

In [8]:
in_vars = ['QBP','TfromTS','PS', 'SOLIN', 'SHFLX', 'LHFLX']
out_vars = ['PHQ','TPHYSTND','FSNT','FSNS','FLNT','FLNS']
train_gen = DataGenerator(
    data_fn = '/local/Tom.Beucler/SPCAM_PHYS/119_train_shuffle.nc',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = '/local/Tom.Beucler/SPCAM_PHYS/118_norm.nc',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True
)

In [10]:
valid_gen = DataGenerator(
    data_fn = '/local/Tom.Beucler/SPCAM_PHYS/119_valid.nc',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = '/local/Tom.Beucler/SPCAM_PHYS/118_norm.nc',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True
)

In [11]:
X, Y = train_gen[25]; X.shape, Y.shape

((1024, 64), (1024, 64))

## 6.2) Model and training

In [12]:
inp6 = Input(shape=(64,))
densout = Dense(128, activation='linear')(inp6)
densout = LeakyReLU(alpha=0.3)(densout)
for i in range (5):
    densout = Dense(128, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
densout = Dense(64, activation='linear')(densout)
out6 = LeakyReLU(alpha=0.3)(densout)
NNmodel6 = tf.keras.models.Model(inp6,out6)

W1116 21:20:54.209091 22502719780672 deprecation.py:506] From /home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [13]:
NNmodel6.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense (Dense)                (None, 128)               8320      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 128)               0     

In [17]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint('/local/Tom.Beucler/SPCAM_PHYS/HDF5_DATA/POG119.hdf5',save_best_only=True, monitor='val_loss', mode='min')

NNmodel6.compile(tf.keras.optimizers.Adam(),loss=mse)

Nep = 15
NNmodel6.fit_generator(train_gen, epochs=Nep, validation_data=valid_gen,callbacks=[earlyStopping,mcp_save])

# 7) NN using RH and (T-Ts) as inputs

In [5]:
!python preprocessing-11132019.py -c /home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/pp_config/122_POG_84G_RHTfromTSinput.yml

/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages

In [6]:
!python preprocessing-11132019.py -c /home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/pp_config/123_POG_84G_p4K_RHTfromTSinput.yml

/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages

In [7]:
in_vars = ['RH','TfromTS','PS', 'SOLIN', 'SHFLX', 'LHFLX']
out_vars = ['PHQ','TPHYSTND','FSNT','FSNS','FLNT','FLNS']
train_gen = DataGenerator(
    data_fn = '/local/Tom.Beucler/SPCAM_PHYS/122_train_shuffle.nc',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = '/local/Tom.Beucler/SPCAM_PHYS/121_norm.nc',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True
)

valid_gen = DataGenerator(
    data_fn = '/local/Tom.Beucler/SPCAM_PHYS/122_valid.nc',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = '/local/Tom.Beucler/SPCAM_PHYS/121_norm.nc',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True
)

In [8]:
X, Y = train_gen[42]; X.shape, Y.shape

((1024, 64), (1024, 64))

In [9]:
inp7 = Input(shape=(64,))
densout = Dense(128, activation='linear')(inp7)
densout = LeakyReLU(alpha=0.3)(densout)
for i in range (5):
    densout = Dense(128, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
densout = Dense(64, activation='linear')(densout)
out7 = LeakyReLU(alpha=0.3)(densout)
NNmodel7 = tf.keras.models.Model(inp7,out7)

W1117 20:11:59.730793 22777608677184 deprecation.py:506] From /home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [10]:
NNmodel7.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense (Dense)                (None, 128)               8320      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 128)               0     

In [11]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint('/local/Tom.Beucler/SPCAM_PHYS/HDF5_DATA/POG122.hdf5',save_best_only=True, monitor='val_loss', mode='min')

NNmodel7.compile(tf.keras.optimizers.Adam(),loss=mse)

Nep = 15
NNmodel7.fit_generator(train_gen, epochs=Nep, validation_data=valid_gen,callbacks=[earlyStopping,mcp_save])

Epoch 1/15
41376/41376 [==============================] - 390s 9ms/step - loss: 573.4809 - val_loss: 537.2603
Epoch 2/15
41376/41376 [==============================] - 300s 7ms/step - loss: 515.8748 - val_loss: 519.6187
Epoch 3/15
41376/41376 [==============================] - 302s 7ms/step - loss: 504.2135 - val_loss: 510.1307
Epoch 4/15
41376/41376 [==============================] - 301s 7ms/step - loss: 498.6930 - val_loss: 505.5444
Epoch 5/15
41376/41376 [==============================] - 304s 7ms/step - loss: 495.2784 - val_loss: 503.1580
Epoch 6/15
41376/41376 [==============================] - 302s 7ms/step - loss: 492.6951 - val_loss: 498.4757
Epoch 7/15
41376/41376 [==============================] - 303s 7ms/step - loss: 490.7393 - val_loss: 500.5692
Epoch 8/15
41376/41376 [==============================] - 302s 7ms/step - loss: 489.2381 - val_loss: 500.7832
Epoch 9/15
41376/41376 [==============================] - 302s 7ms/step - loss: 488.0370 - val_loss: 495.4300
Epoch 10/1

# 7) NN using RH and Carnotmax as inputs

In [12]:
!python preprocessing-11132019.py -c /home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/pp_config/125_POG_84G_RHCarnotmaxinput.yml

/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages

In [13]:
!python preprocessing-11132019.py -c /home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/pp_config/126_POG_84G_p4K_RHCarnotmaxinput.yml

/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages

In [14]:
in_vars = ['RH','Carnotmax','PS', 'SOLIN', 'SHFLX', 'LHFLX']
out_vars = ['PHQ','TPHYSTND','FSNT','FSNS','FLNT','FLNS']
train_gen = DataGenerator(
    data_fn = '/local/Tom.Beucler/SPCAM_PHYS/125_train_shuffle.nc',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = '/local/Tom.Beucler/SPCAM_PHYS/124_norm.nc',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True
)

valid_gen = DataGenerator(
    data_fn = '/local/Tom.Beucler/SPCAM_PHYS/125_valid.nc',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = '/local/Tom.Beucler/SPCAM_PHYS/124_norm.nc',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True
)

In [15]:
X, Y = train_gen[12]; X.shape, Y.shape

((1024, 64), (1024, 64))

In [16]:
inp8 = Input(shape=(64,))
densout = Dense(128, activation='linear')(inp8)
densout = LeakyReLU(alpha=0.3)(densout)
for i in range (5):
    densout = Dense(128, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
densout = Dense(64, activation='linear')(densout)
out8 = LeakyReLU(alpha=0.3)(densout)
NNmodel8 = tf.keras.models.Model(inp8,out8)

In [17]:
NNmodel8.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense_7 (Dense)              (None, 128)               8320      
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 128)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 128)               16512     
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, 128)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 128)               16512     
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 128)               0   

In [18]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint('/local/Tom.Beucler/SPCAM_PHYS/HDF5_DATA/POG125.hdf5',save_best_only=True, monitor='val_loss', mode='min')

NNmodel8.compile(tf.keras.optimizers.Adam(),loss=mse)

Nep = 15
NNmodel8.fit_generator(train_gen, epochs=Nep, validation_data=valid_gen,callbacks=[earlyStopping,mcp_save])

Epoch 1/15
41376/41376 [==============================] - 407s 10ms/step - loss: 583.7191 - val_loss: 546.7006
Epoch 2/15
41376/41376 [==============================] - 314s 8ms/step - loss: 526.4531 - val_loss: 530.3767
Epoch 3/15
41376/41376 [==============================] - 321s 8ms/step - loss: 514.0157 - val_loss: 520.1909
Epoch 4/15
41376/41376 [==============================] - 322s 8ms/step - loss: 507.8843 - val_loss: 514.9544
Epoch 5/15
41376/41376 [==============================] - 300s 7ms/step - loss: 504.0550 - val_loss: 512.4326
Epoch 6/15
41376/41376 [==============================] - 299s 7ms/step - loss: 501.2051 - val_loss: 509.0269
Epoch 7/15
41376/41376 [==============================] - 300s 7ms/step - loss: 499.0008 - val_loss: 509.0828
Epoch 8/15
41376/41376 [==============================] - 301s 7ms/step - loss: 497.3278 - val_loss: 505.5856
Epoch 9/15
41376/41376 [==============================] - 301s 7ms/step - loss: 495.8387 - val_loss: 505.5064
Epoch 10/